In [1]:
import pandas as pd
import numpy as np
#pd.options.display.float_format = '{:.2f}'.format

std = 40
std90 = std*0.9

In [62]:
def readSaturation(name):
    df=pd.read_csv("포화도(40명).csv",encoding='UTF8')
    df=df.drop(['Unnamed: 0'],axis=1)
    df=df.drop(['지역'],axis=1)
    df=df.drop(['일시'],axis=1)
    df=df.drop(['월'],axis=1)
    df=df.drop(['하차'],axis=1)
    df=df.drop(['전체'],axis=1)

    for i,row in df.iterrows():    

        #왕복 운행 대수
        #편도 운행시간 60분 이상 노선들
        if df.at[i,'노선운행시간']>=60:
            df.at[i,'왕복운행대수']=round((df.at[i,'노선운행시간']/df.at[i,'주중 평균배차시간']*2),2)
        #편도 운행시간 60분 이하 노선들
        else:
            df.at[i,'왕복운행대수']=round((60/df.at[i,'주중 평균배차시간']*2),2)
    return df

In [70]:
#동적 배차시간 만들기
def dynamicTime(df):
    dfU=df[df['시간별 포화도']>=100]
    dfU=dfU.reset_index(drop=True) 

    #포화도를 90퍼센트까지 내리는 것으로 목표
    for i,row in dfU.iterrows():    

        #단순히 증차한 운행대수 (승차 인원/포화도90기준차량승객수)
        dfU.at[i,'증차운행대수']= round((dfU.at[i,'승차']/std90),2)

        #증차를 기존 운행대수 130%로 제한한 현실적인 운행대수
        if dfU.at[i,'증차운행대수']>= dfU.at[i,'왕복운행대수']*1.3:
            dfU.at[i,'증차운행대수']=round((dfU.at[i,'왕복운행대수']*1.3),2)

        #증차 대수
        dfU.at[i,'증차대수'] = dfU.at[i,'증차운행대수']-dfU.at[i,'왕복운행대수']

        #증차 이후 배차시간
        if dfU.at[i,'노선운행시간']>=60:
            dfU.at[i, '증차후배차시간'] = round((dfU.at[i,'노선운행시간']/dfU.at[i,'증차운행대수']*2),2)
        #편도 운행시간 60분 이하 노선들
        else:
            dfU.at[i, '증차후배차시간'] = round((60/dfU.at[i,'증차운행대수']*2),2)


        #증차 이후 예상 포화도
        dfU.at[i,'증차후예상포화도'] = round((dfU.at[i,'승차']/(dfU.at[i,'증차운행대수']*std)*100),2)

        #감소한 포화도
        dfU.at[i, '포화도감소율'] = dfU.at[i,'시간별 포화도']-dfU.at[i,'증차후예상포화도']
    return dfU

In [72]:
#증차 대체 감차 운행시간 찾기
#기존 데이터와 동적 배차시간 데이터 필요
def compare(df,dfU):
    dfL=dfU.copy()
    dfL=dfL.drop(['승차'],axis=1)
    dfL=dfL.drop(['시·종점'],axis=1)
    dfL=dfL.drop(['포화도감소율'],axis=1)
    
    for i,row in dfL.iterrows():
        busNum=dfL.at[i,'노선']
        
        #포화도 높은 순과 낮은 순의 데이터를 정렬하여 1대1로 증차와 감차
        #포화도 100이 넘어가는 중복되는 노선의 순서, 순서는 0에 가까울수록 포화도가 높음
        #포화도 순서 찾기
        count=0
        for j,jrow in dfL.iterrows():
            if i==j: break
            if dfL.at[i,'노선']==dfL.at[j,'노선']: count=count+1
        #노선의 감차 시간대 데이터 찾아서 리스트에 추가
        #가장 승차수 적은 시간대 찾기(7시 ~ 22시 사이)
        #이외의 시간들은 종료 시점이라 의미 없을 가능성 多
        leastlist=[]
        for a in range(12):
            #시간별 포화도의 오름차순으로 정렬해서 포화도 낮은순으로 리스트에 추가
            l=df[df['노선']==busNum].sort_values(by=['시간별 포화도'],axis=0,ascending=True).iloc[a]
            if l['시간'] not in ['22시~23시','23시~24시','00시~01시','01시~02시','02시~03시','03시~04시','04시~05시','05시~06시','06시~07시']:
                leastlist.append(l)
                
        #감차할 시간대 데이터
        least=leastlist[count]
        
        
        #노선 감차 시간대 데이터 dfL에 추가
        dfL.at[i,'감차대상시간']=least['시간']
        dfL.at[i,'감차대상포화도']=least['시간별 포화도']
        dfL.at[i,'감차운행대수']=dfL.at[i,'왕복운행대수']-dfL.at[i,'증차대수']

        #감차후배차시간 계산
        if dfL.at[i,'노선운행시간']>=60:
            dfL.at[i,'감차후배차시간']=round((dfL.at[i,'노선운행시간']/dfL.at[i,'감차운행대수']*2),2)
        #편도 운행시간 60분 이하 노선들
        else:
            dfL.at[i,'감차후배차시간'] = round((60/dfL.at[i,'감차운행대수']*2),2)

        #감차 이후 예상 포화도
        dfL.at[i,'감차후예상포화도'] = round((least['승차']/(dfL.at[i,'감차운행대수']*std)*100),2)
        
    #노선 번호로 Sort
    dfL=dfL.sort_values(by=['노선'],axis=0,ascending = True)
    return dfL

In [50]:
def toCsv(name, df):
    tmp = df.copy()
    for i,row in tmp.iterrows():
        #저장후 노선 번호 이상문제 ' '추가로 해결
        tmp.at[i,'노선'] = ' '+tmp.at[i,'노선']
        
    tmp.to_csv(name, encoding ='cp949', index = None)

In [64]:
df=readSaturation("포화도 계산 21년 4월.csv")
df

,노선,시·종점,시간,승차,시간별 포화도,주중 평균배차시간,노선운행시간,왕복운행대수
0,20-1,정왕역환승센터-부곡중학교,07시~08시,761,183,15.0,78,10.40
1,23,월곶역-본오동차고지,18시~19시,1072,169,12.5,99,15.84
2,39,매화초등학교-개봉역,07시~08시,386,165,20.5,45,5.85
3,11,광명돔경륜장.광남문-안양역,08시~09시,262,164,32.5,65,4.00
4,12,삼미시장-김포공항국내선(9번홈),08시~09시,585,164,15.0,67,8.93
...,...,...,...,...,...,...,...,...
1235,55,본오동종점-오이도차고지,00시~01시,0,0,9.0,93,20.67
1236,98,오이도차고지-수변공원휴게소,00시~01시,0,0,27.5,104,7.56
1237,23-2,소사공영차고지-라일락마을어린이공원,00시~01시,2,0,8.5,69,16.24
1238,G8808,송내역남부(광역버스)-KTX광명역5번출구,22시~23시,0,0,30.0,31,4.00


In [71]:
dfU=dynamicTime(df)
toCsv("4월 동적 배차시간.csv",dfU)
dfU

,노선,시·종점,시간,승차,시간별 포화도,주중 평균배차시간,노선운행시간,왕복운행대수,증차운행대수,증차대수,증차후배차시간,증차후예상포화도,포화도감소율
0,20-1,정왕역환승센터-부곡중학교,07시~08시,761,183,15.0,78,10.40,13.52,3.12,11.54,140.72,42.28
1,23,월곶역-본오동차고지,18시~19시,1072,169,12.5,99,15.84,20.59,4.75,9.62,130.16,38.84
2,39,매화초등학교-개봉역,07시~08시,386,165,20.5,45,5.85,7.60,1.75,15.79,126.97,38.03
3,11,광명돔경륜장.광남문-안양역,08시~09시,262,164,32.5,65,4.00,5.20,1.20,25.00,125.96,38.04
4,12,삼미시장-김포공항국내선(9번홈),08시~09시,585,164,15.0,67,8.93,11.61,2.68,11.54,125.97,38.03
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,20,경원여객-시흥시청후문.종합일자리센터,08시~09시,69,101,70.0,40,1.71,1.92,0.21,62.50,89.84,11.16
66,81,목감호수품애.중흥S클래스-안양역,08시~09시,368,100,13.0,39,9.23,10.22,0.99,11.74,90.02,9.98
67,55,본오동종점-오이도차고지,18시~19시,824,100,9.0,93,20.67,22.89,2.22,8.13,90.00,10.00
68,32,경원여객안양차고지-풍림2단지.소래포구입구,18시~19시,275,100,25.0,86,6.88,7.64,0.76,22.51,89.99,10.01


In [73]:
dfL=compare(df,dfU)
toCsv("4월 증차, 감차 대상 비교 분석.csv",dfL)
dfL

,노선,시간,시간별 포화도,주중 평균배차시간,노선운행시간,왕복운행대수,증차운행대수,증차대수,증차후배차시간,증차후예상포화도,감차대상시간,감차대상포화도,감차운행대수,감차후배차시간,감차후예상포화도
25,1(광명),08시~09시,125,8.5,108,25.41,33.03,7.62,6.54,96.20,11시~12시,61.0,17.79,12.14,86.85
18,1(광명),18시~19시,135,8.5,108,25.41,33.03,7.62,6.54,103.77,21시~22시,57.0,17.79,12.14,81.51
30,1(광명),17시~18시,119,8.5,108,25.41,33.03,7.62,6.54,91.73,12시~13시,68.0,17.79,12.14,96.96
59,1(광명),16시~17시,102,8.5,108,25.41,28.78,3.37,7.51,89.99,13시~14시,69.0,22.04,9.80,79.29
7,1(광명),07시~08시,151,8.5,108,25.41,33.03,7.62,6.54,116.03,10시~11시,56.0,17.79,12.14,79.96
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23,76,07시~08시,126,90.0,86,1.91,2.48,0.57,69.35,96.77,16시~17시,25.0,1.34,128.36,35.45
33,81,07시~08시,118,13.0,39,9.23,12.00,2.77,10.00,91.04,11시~12시,42.0,6.46,18.58,59.60
66,81,08시~09시,100,13.0,39,9.23,10.22,0.99,11.74,90.02,14시~15시,48.0,8.24,14.56,54.31
54,98,07시~08시,106,27.5,104,7.56,8.89,1.33,23.40,89.99,11시~12시,33.0,6.23,33.39,40.13
